In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema
from datetime import datetime

In [2]:
data = pd.read_csv("/Users/yuxiding/Desktop/dataset.s010", sep="\t")

In [3]:
diabetic = data[data['subjectId'] == "1636-69-001"]
prediabetic = data[data['subjectId'] == "1636-69-114"]
non_diabetic = data[data['subjectId'] == "1636-69-028"]

In [4]:
diabetic = diabetic.iloc[:, 0:3]
diabetic

,DisplayTime,GlucoseValue,subjectId
0,2014-02-03 03:42:12,93,1636-69-001
1,2014-02-03 03:47:12,93,1636-69-001
2,2014-02-03 03:52:12,93,1636-69-001
3,2014-02-03 03:57:12,95,1636-69-001
4,2014-02-03 04:02:12,96,1636-69-001
...,...,...,...
1841,2015-04-02 14:48:06,115,1636-69-001
1842,2015-04-02 14:53:06,114,1636-69-001
1843,2015-04-02 14:58:06,117,1636-69-001
1844,2015-04-02 15:03:06,121,1636-69-001


In [5]:
#mean
def meangv(gvdata, unit = 'mg'):
    return np.average(gvdata['GlucoseValue'].astype(float))

In [6]:
meangv(diabetic)

108.22860238353196

In [104]:
#mean value
def mvalue(gvdata, referece_value = 90, unit = 'mg'):
    gvdata['date'] = gvdata['DisplayTime']
    
    fmt_1 = '%Y-%m-%d %H:%M:%S'
    fmt_2 = '%Y-%m-%d'
    fmt_3 = '%Y-%m-%d %H:%M'
    
    
    for i in range(len(gvdata)):
        gvdata['date'][i] = datetime.strptime((gvdata['DisplayTime']).iloc[i], fmt_1).strftime(fmt_2)
    
    dict_of_date = {k: v for k, v in gvdata.groupby('date')}
    
        
    day1 = datetime.strptime((datetime.strptime(gvdata['DisplayTime'][0], fmt_1)).strftime(fmt_3), fmt_3)
    day2 = datetime.strptime((datetime.strptime(gvdata['DisplayTime'][len(gvdata['DisplayTime'])-1], fmt_1)).strftime(fmt_3), fmt_3)
            
    diff = day2 - day1           
    days = diff.days
                
    
    m_gr_list = []
    m_w_list = []
    m_list = []
         
    for j in range(len(dict_of_date)):
        
        key = list(dict_of_date.keys())[j]
        value = dict_of_date.get(key)
        
        #unit
        if (unit == 'mmol'):
            value['GlucoseValue'] = (value['GlucoseValue']).astype(float) / 18
        
        #m_w
        m_w = (float(np.max(value['GlucoseValue'])) - float(np.min(value['GlucoseValue']))) / 20
        m_w_list.append(m_w)
        
        #m_gr
        for m in value['GlucoseValue']:
            m_gr = np.sum((np.absolute(10*np.log(float(m)/referece_value)))**3) / len(value['GlucoseValue'])
        
        m_gr_list.append(m_gr)
        
        if days < 1:
            m_list.append(m_gr_list[j] + m_w_list[j])
        else:
            m_list.append(m_gr_list[j])
            
            
    mvalue = np.average(m_list)
                  
    return mvalue
        
    

In [105]:
print(mvalue(diabetic, unit = 'mg'))
print(mvalue(diabetic, unit = 'mmol'))

0.1536879044196218
102.36108090147624


In [9]:
#J-index
def jindex(gvdata, unit = 'mg'):
    
    if (unit == 'mmol'):
        gvdata['GlucoseValue'] = (gvdata['GlucoseValue']).astype(float) / 18
        
    mbg = np.average(gvdata['GlucoseValue'].astype(float))
    sd = np.std(gvdata['GlucoseValue'].astype(float))
    j = 0.324 * (mbg + sd) ** 2
    return j

In [10]:
print(jindex(diabetic, unit = 'mg'))
print(jindex(diabetic, unit = 'mmol'))

5950.7901669300545
18.366636317685355


In [91]:
def modd(gvdata, unit = 'mg', hour_ago = 24):
    
    if (unit == 'mmol'):
        gvdata['GlucoseValue'] = (gvdata['GlucoseValue']).astype(float) / 18

    gvdata['date'] = gvdata['DisplayTime']
    gvdata['time'] = gvdata['DisplayTime']
    
    fmt_1 = '%Y-%m-%d %H:%M:%S'
    fmt_2 = '%Y-%m-%d'
    fmt_3 = '%Y-%m-%d %H:%M'
    
    for i in range(len(gvdata)):
        gvdata['date'][i] = datetime.strptime((gvdata['DisplayTime']).iloc[i], fmt_1).strftime(fmt_2)
    
    dict_of_date = {k: v for k, v in gvdata.groupby('date')}

    modd_list = []
    for j in range(len(dict_of_date)-1):
        
        key_day1 = list(dict_of_date.keys())[j]
        value_day1 = dict_of_date.get(key_day1)
    
        
        key_day2 = list(dict_of_date.keys())[j+1]
        value_day2 = dict_of_date.get(key_day2)
    
        gr_list = []
        index_day1 = []
        index_day2 = []
    
        for k in value_day1['DisplayTime']:
            for m in value_day2['DisplayTime']:
                day1 = datetime.strptime((datetime.strptime(k, fmt_1)).strftime(fmt_3), fmt_3)
                day2 = datetime.strptime((datetime.strptime(m, fmt_1)).strftime(fmt_3), fmt_3)
            
                diff = day2 - day1
            
                days = diff.days
                secs = diff.seconds

                if days == 1 and secs == 0:

                    index_day1.append((value_day1['DisplayTime'].index[value_day1.DisplayTime == k])[0])
                    index_day2.append((value_day2['DisplayTime'].index[value_day2.DisplayTime == m])[0])
                
                
        for n in range(len(list(index_day1))):
            gr_list.append(np.absolute(float(value_day1['GlucoseValue'][index_day1[n]]) - float(value_day2['GlucoseValue'][index_day2[n]])))
        modd_list.append(np.average(gr_list))   

    modd_list = [modd for modd in modd_list if str(modd) != 'nan']
    modd_value = np.average(modd_list)
    
    return modd_value



In [92]:
modd(diabetic)

24.629513396318345

In [111]:
def conga(gvdata, n = 1, unit = 'mg'):
    
    if (unit == 'mmol'):
        gvdata['GlucoseValue'] = (gvdata['GlucoseValue']).astype(float) / 18
        
    gv = gvdata['GlucoseValue']
    diff = []
    for i in range(len(gvdata)):
        for j in range(i+1, len(gvdata)):
            fmt_1 = '%Y-%m-%d %H:%M:%S'
            fmt_2 = '%Y-%m-%d %H:%M'
            
            time = gvdata['DisplayTime']
            t1 = datetime.strptime((datetime.strptime(time.iloc[i], fmt_1)).strftime(fmt_2), fmt_2)
            t2 = datetime.strptime((datetime.strptime(time.iloc[j], fmt_1)).strftime(fmt_2), fmt_2)
            tdelta = t2 - t1
            t_diff = tdelta.seconds
            
            if t_diff == n*60*60:
                diff.append(int(gv.iloc[j]) - int(gv.iloc[i]))
            std_diff = np.std(diff, axis=0)
    print(std_diff)
            
    
    
    

In [112]:
conga(diabetic)

/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


33.33609739776171


In [115]:
def grade(gvdata, n = 1, unit = 'mg'):
    if (unit == 'mmol'):
        gvdata['GlucoseValue'] = (gvdata['GlucoseValue']).astype(float) / 18
        
    grade_list = []
    for x in gvdata['GlucoseValue']:
        grade = 425 * (np.log(np.log(float(x))) + 0.16) ** 2
        grade_list.append(grade)
        
    grade_value = np.average(grade_list)
    return grade_value
    

In [116]:
grade(diabetic)

1225.2193158838713